<a href="https://colab.research.google.com/github/ivan-rib/dados_fraude_cartao/blob/main/fraude_cartao_credito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalando o spark e craiando uma spark session

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [5]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 17.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=89115990b6b579422bc7935b5eaebdf870e1037c23ba80e1159c6dad27dd7813
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [12]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

## lendo arquivo csv utilizando pyspark

In [6]:
#lendo csv do primeiro dataset
df_test = spark.read.format('csv') \
  .option("header","true")\
	.option("delimiter",",")\
  .option("encoding", "ISO-8859-1")\
	.option("inferSchema","true")\
  .load('/content/fraudTest.csv')

In [7]:
df_test.show(5, truncate = False)

+---+---------------------+----------------+------------------------------------+--------------+-----+------+--------+------+---------------------------+----------+-----+-----+-------+------------------+--------+----------------------+-------------------+--------------------------------+----------+------------------+-----------+--------+
|_c0|trans_date_trans_time|cc_num          |merchant                            |category      |amt  |first |last    |gender|street                     |city      |state|zip  |lat    |long              |city_pop|job                   |dob                |trans_num                       |unix_time |merch_lat         |merch_long |is_fraud|
+---+---------------------+----------------+------------------------------------+--------------+-----+------+--------+------+---------------------------+----------+-----+-----+-------+------------------+--------+----------------------+-------------------+--------------------------------+----------+------------------+--

In [18]:
# contando a quantidade de linhas do dataset
df_test.count()

399289

# aplicando uma consulta usando select e where

In [32]:
df_test.select(col('first'),col('last'),col('trans_num')).where(df_test.trans_num == '2da90c7d74bd46a0caf3777415b3ebd3').show()

+-----+-------+--------------------+
|first|   last|           trans_num|
+-----+-------+--------------------+
| Jeff|Elliott|2da90c7d74bd46a0c...|
+-----+-------+--------------------+



In [8]:
# lendo csv do segundo dataset
df_train = spark.read.format('csv') \
  .option("header","true")\
	.option("delimiter",",")\
  .option("encoding", "ISO-8859-1")\
	.option("inferSchema","true")\
  .load('/content/fraudTrain.csv')

In [9]:
df_train.show(5, truncate = False)

+---+---------------------+----------------+----------------------------------+-------------+------+---------+-------+------+----------------------------+--------------+-----+-----+-------+---------+--------+---------------------------------+-------------------+--------------------------------+----------+------------------+-----------+--------+
|_c0|trans_date_trans_time|cc_num          |merchant                          |category     |amt   |first    |last   |gender|street                      |city          |state|zip  |lat    |long     |city_pop|job                              |dob                |trans_num                       |unix_time |merch_lat         |merch_long |is_fraud|
+---+---------------------+----------------+----------------------------------+-------------+------+---------+-------+------+----------------------------+--------------+-----+-----+-------+---------+--------+---------------------------------+-------------------+--------------------------------+----------+

In [37]:
## verificando se os dados do primeiro dataset aparecem no segundo
df_train.select('first','last','trans_num').where(df_train.trans_num == '2da90c7d74bd46a0caf3777415b3ebd3').show()

+-----+----+---------+
|first|last|trans_num|
+-----+----+---------+
+-----+----+---------+



# comparando os schemas dos datasets

In [35]:
df_test.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: timestamp (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- unix_time: integer (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)



In [36]:
df_train.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: timestamp (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- unix_time: integer (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)



#concatenando dois arquivos csv

In [38]:
df_final = df_train.union(df_test)

In [39]:
df_final.count()

887691

In [40]:
df_final.show(5)

+---+---------------------+----------------+--------------------+-------------+------+---------+-------+------+--------------------+--------------+-----+-----+-------+---------+--------+--------------------+-------------------+--------------------+----------+------------------+-----------+--------+
|_c0|trans_date_trans_time|          cc_num|            merchant|     category|   amt|    first|   last|gender|              street|          city|state|  zip|    lat|     long|city_pop|                 job|                dob|           trans_num| unix_time|         merch_lat| merch_long|is_fraud|
+---+---------------------+----------------+--------------------+-------------+------+---------+-------+------+--------------------+--------------+-----+-----+-------+---------+--------+--------------------+-------------------+--------------------+----------+------------------+-----------+--------+
|  0|  2019-01-01 00:00:18|2703186189652095|fraud_Rippin, Kub...|     misc_net|  4.97| Jennifer|  Ba

# transformação dos dados

In [43]:
# excluindo algumas colunas
df_final = df_final.drop('_c0','lat','long','merch_lat','merch_long','unix_time')

In [44]:
df_final.show(5)

+---------------------+----------------+--------------------+-------------+------+---------+-------+------+--------------------+--------------+-----+-----+--------+--------------------+-------------------+--------------------+--------+
|trans_date_trans_time|          cc_num|            merchant|     category|   amt|    first|   last|gender|              street|          city|state|  zip|city_pop|                 job|                dob|           trans_num|is_fraud|
+---------------------+----------------+--------------------+-------------+------+---------+-------+------+--------------------+--------------+-----+-----+--------+--------------------+-------------------+--------------------+--------+
|  2019-01-01 00:00:18|2703186189652095|fraud_Rippin, Kub...|     misc_net|  4.97| Jennifer|  Banks|     F|      561 Perry Cove|Moravian Falls|   NC|28654|    3495|Psychologist, cou...|1988-03-09 00:00:00|0b242abb623afc578...|       0|
|  2019-01-01 00:00:44|    630423337322|fraud_Heller, Gu

In [68]:
#alterando nomes das colunas
df_final = df_final.withColumnRenamed('trans_date_trans_time', 'data_transacao') \
          .withColumnRenamed('cc_num', 'codigo_cartao') \
          .withColumnRenamed('merchant', 'estabelecimento')\
          .withColumnRenamed('category', 'categoria') \
          .withColumnRenamed('amt', 'valor') \
          .withColumnRenamed('first','primeiro_nome') \
          .withColumnRenamed('last', 'sobrenome') \
          .withColumnRenamed('gender','genero') \
          .withColumnRenamed('street', 'endereco') \
          .withColumnRenamed('city', 'cidade') \
          .withColumnRenamed('zip','cep') \
          .withColumnRenamed('state','estado') \
          .withColumnRenamed('city_pop', 'numero') \
          .withColumnRenamed('job','profissao') \
          .withColumnRenamed('dob','data_nasc') \
          .withColumnRenamed('trans_num','numero_transacao') \
          .withColumnRenamed('is_fraud','fraude')


In [69]:
df_final.show(5)

+-------------------+----------------+--------------------+-------------+------+-------------+---------+------+--------------------+--------------+------+-----+------+--------------------+-------------------+--------------------+------+--------------+----------+-----------+---------------+
|     data_transacao|   codigo_cartao|     estabelecimento|    categoria| valor|primeiro_nome|sobrenome|genero|            endereco|        cidade|estado|  cep|numero|           profissao|          data_nasc|    numero_transacao|fraude|hora_transacao|      data|data_compra|data_nascimento|
+-------------------+----------------+--------------------+-------------+------+-------------+---------+------+--------------------+--------------+------+-----+------+--------------------+-------------------+--------------------+------+--------------+----------+-----------+---------------+
|2019-01-01 00:00:18|2703186189652095|fraud_Rippin, Kub...|     misc_net|  4.97|     Jennifer|    Banks|     F|      561 Perry 

#separando coluna data_operação e dob em duas


In [67]:
# criando a variavel que recebe o split(separação)
split_cols = split(df_final['data_transacao'], ' ')
split_cols1 = split(df_final['data_nasc'], ' ')
  
#separando os dados das colunas data_transacao e dob
df_final = df_final.withColumn('hora_transacao', split_cols.getItem(1)) \
            .withColumn('data_compra', split_cols.getItem(0)) \
            .withColumn('data_nascimento', split_cols1.getItem(0))
    
  
# show df
df_final.show(5)

+-------------------+----------------+--------------------+-------------+------+-------------+---------+------+--------------------+--------------+------+-----+------+--------------------+-------------------+--------------------+------+--------------+----------+-----------+---------------+
|     data_transacao|   codigo_cartao|     estabelecimento|    categoria| valor|primeiro_nome|sobrenome|genero|            endereco|        cidade|estado|  cep|numero|           profissao|          data_nasc|    numero_transacao|fraude|hora_transacao|      data|data_compra|data_nascimento|
+-------------------+----------------+--------------------+-------------+------+-------------+---------+------+--------------------+--------------+------+-----+------+--------------------+-------------------+--------------------+------+--------------+----------+-----------+---------------+
|2019-01-01 00:00:18|2703186189652095|fraud_Rippin, Kub...|     misc_net|  4.97|     Jennifer|    Banks|     F|      561 Perry 

In [70]:
#excluindo colunas que foram separadas em outras
df_final = df_final.drop('data_transacao','data_nasc','data')

In [71]:
df_final.show(5)

+----------------+--------------------+-------------+------+-------------+---------+------+--------------------+--------------+------+-----+------+--------------------+--------------------+------+--------------+-----------+---------------+
|   codigo_cartao|     estabelecimento|    categoria| valor|primeiro_nome|sobrenome|genero|            endereco|        cidade|estado|  cep|numero|           profissao|    numero_transacao|fraude|hora_transacao|data_compra|data_nascimento|
+----------------+--------------------+-------------+------+-------------+---------+------+--------------------+--------------+------+-----+------+--------------------+--------------------+------+--------------+-----------+---------------+
|2703186189652095|fraud_Rippin, Kub...|     misc_net|  4.97|     Jennifer|    Banks|     F|      561 Perry Cove|Moravian Falls|    NC|28654|  3495|Psychologist, cou...|0b242abb623afc578...|     0|      00:00:18| 2019-01-01|     1988-03-09|
|    630423337322|fraud_Heller, Gut...| 

In [76]:
df_final.select('hora_transacao','estabelecimento','categoria','fraude').where(df_final.fraude == 1).show(5)

+--------------+--------------------+-------------+------+
|hora_transacao|     estabelecimento|    categoria|fraude|
+--------------+--------------------+-------------+------+
|      01:06:37|fraud_Rutherford-...|  grocery_pos|     1|
|      01:47:29|fraud_Jenkins, Ha...|gas_transport|     1|
|      03:05:23|fraud_Goodwin-Nit...|  grocery_pos|     1|
|      03:38:03|fraud_Erdman-Kert...|gas_transport|     1|
|      03:55:47|  fraud_Koepp-Parker|  grocery_pos|     1|
+--------------+--------------------+-------------+------+
only showing top 5 rows



# vendo quantos valores distintos existem dentro da coluna

In [92]:
df_final.select(countDistinct('fraude')).show()


+----------------------+
|count(DISTINCT fraude)|
+----------------------+
|                     2|
+----------------------+



In [133]:
#alterando valores dentros da coluna fraude
data_ready = df_final.withColumn('fraude', regexp_replace('fraude', '0', 'nao')) \
                    .withColumn('fraude', regexp_replace('fraude', '1', 'sim'))

In [105]:
data_ready.show(5)

+----------------+--------------------+-------------+------+-------------+---------+------+--------------------+--------------+------+-----+------+--------------------+--------------------+------+--------------+-----------+---------------+
|   codigo_cartao|     estabelecimento|    categoria| valor|primeiro_nome|sobrenome|genero|            endereco|        cidade|estado|  cep|numero|           profissao|    numero_transacao|fraude|hora_transacao|data_compra|data_nascimento|
+----------------+--------------------+-------------+------+-------------+---------+------+--------------------+--------------+------+-----+------+--------------------+--------------------+------+--------------+-----------+---------------+
|2703186189652095|fraud_Rippin, Kub...|     misc_net|  4.97|     Jennifer|    Banks|     F|      561 Perry Cove|Moravian Falls|    NC|28654|  3495|Psychologist, cou...|0b242abb623afc578...|   sim|      00:00:18| 2019-01-01|     1988-03-09|
|    630423337322|fraud_Heller, Gut...| 

# criando uma consulta sql com spark

In [109]:
data_ready.createTempView("data_ready")

In [123]:
spark.sql("")("select * from data_ready where fraude = 'nao' limit 5").show()

+---------------+--------------------+-------------+------+-------------+---------+------+--------------------+-------------+------+-----+-------+--------------------+--------------------+------+--------------+-----------+---------------+
|  codigo_cartao|     estabelecimento|    categoria| valor|primeiro_nome|sobrenome|genero|            endereco|       cidade|estado|  cep| numero|           profissao|    numero_transacao|fraude|hora_transacao|data_compra|data_nascimento|
+---------------+--------------------+-------------+------+-------------+---------+------+--------------------+-------------+------+-----+-------+--------------------+--------------------+------+--------------+-----------+---------------+
|  4613314721966|fraud_Rutherford-...|  grocery_pos|281.06|        Jason|   Murphy|     M|542 Steve Curve S...|Collettsville|    NC|28611|    885|      Soil scientist|e8a81877ae9a0a7f8...|   nao|      01:06:37| 2019-01-02|     1988-09-15|
|340187018810220|fraud_Jenkins, Ha...|gas_tr

# criando data frames para compras fraudulentas e não fraudulentas

In [134]:
compras_fraudulentas = data_ready.select('codigo_cartao','estabelecimento',
                                         'valor','primeiro_nome','sobrenome','endereco','cidade',
                                         'numero_transacao','hora_transacao','data_compra') \
                      .where(data_ready.fraude == 'sim')

compras_fraudulentas.show(5, truncate =False)

+---------------+--------------------------------+------+-------------+---------+-------------------------+-------------+--------------------------------+--------------+-----------+
|codigo_cartao  |estabelecimento                 |valor |primeiro_nome|sobrenome|endereco                 |cidade       |numero_transacao                |hora_transacao|data_compra|
+---------------+--------------------------------+------+-------------+---------+-------------------------+-------------+--------------------------------+--------------+-----------+
|4613314721966  |fraud_Rutherford-Mertz          |281.06|Jason        |Murphy   |542 Steve Curve Suite 011|Collettsville|e8a81877ae9a0a7f883e15cb39dc4022|01:06:37      |2019-01-02 |
|340187018810220|fraud_Jenkins, Hauck and Friesen|11.52 |Misty        |Hart     |27954 Hall Mill Suite 575|San Antonio  |bc7d41c41103877b03232f03f1f8d3f5|01:47:29      |2019-01-02 |
|340187018810220|fraud_Goodwin-Nitzsche          |276.31|Misty        |Hart     |27954 Hal

In [135]:
compras_nao_fraudulentas = data_ready.select('codigo_cartao','estabelecimento',
                                         'valor','primeiro_nome','sobrenome','endereco','cidade',
                                         'numero_transacao','hora_transacao','data_compra') \
                      .where(data_ready.fraude == 'nao')

compras_nao_fraudulentas.show(5, truncate = False)

+----------------+----------------------------------+------+-------------+---------+----------------------------+--------------+--------------------------------+--------------+-----------+
|codigo_cartao   |estabelecimento                   |valor |primeiro_nome|sobrenome|endereco                    |cidade        |numero_transacao                |hora_transacao|data_compra|
+----------------+----------------------------------+------+-------------+---------+----------------------------+--------------+--------------------------------+--------------+-----------+
|2703186189652095|fraud_Rippin, Kub and Mann        |4.97  |Jennifer     |Banks    |561 Perry Cove              |Moravian Falls|0b242abb623afc578575680df30655b9|00:00:18      |2019-01-01 |
|630423337322    |fraud_Heller, Gutmann and Zieme   |107.23|Stephanie    |Gill     |43039 Riley Greens Suite 393|Orient        |1f76529f8574734946361c461b024d99|00:00:44      |2019-01-01 |
|38859492057661  |fraud_Lind-Buckridge              |22

In [136]:
compras_nao_fraudulentas.count()

882898

In [137]:
compras_fraudulentas.count()

4793